In [1]:
import numpy as np 
from nltk.tokenize import RegexpTokenizer
from keras.models import Sequential, load_model 
from keras.layers import LSTM 
from keras.layers import Dense, Activation
from tensorflow.keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import pickle
import heapq

In [3]:
text=open('1661-0.txt',encoding='UTF-8').read().lower()
print("cropus lenght:",len(text))

cropus lenght: 581888


In [5]:
tokenizer = RegexpTokenizer(r'\w+')
words = tokenizer.tokenize(text)

In [6]:
unique_words = np.unique(words)
unique_words_index = dict((c,i) for i, c in enumerate(unique_words))

In [7]:
WORD_LENGHT = 5
prev_words = []
next_words = []
for i in range (len(words) - WORD_LENGHT):
    prev_words.append(words[i:i + WORD_LENGHT])
    next_words.append(words[i + WORD_LENGHT])
print(prev_words[0])
print(next_words[0])

['project', 'gutenberg', 's', 'the', 'adventures']
of


In [8]:
X = np.zeros((len(prev_words), WORD_LENGHT, len(unique_words)), dtype= bool)
Y = np.zeros((len(next_words), len(unique_words)), dtype= bool)
for i, each_words in enumerate(prev_words):
    for j, each_words in enumerate(each_words):
        X[i,j, unique_words_index[each_words]]=1
    Y[i, unique_words_index[next_words[i]]]=1

In [9]:
print(X[0][0])

[False False False ... False False False]


In [17]:
model = Sequential()
model.add(LSTM(128, input_shape=(WORD_LENGHT, len(unique_words))))
model.add(Dense(len(unique_words)))
model.add(Activation('softmax'))

In [22]:
optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy',optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X, Y, validation_split=0.05, batch_size=128, epochs=2, shuffle=True).history 

Epoch 1/2
811/811 ━━━━━━━━━━━━━━━━━━━━ 140s 170ms/step - accuracy: 0.0576 - loss: 6.6002 - val_accuracy: 0.0850 - val_loss: 6.8780
Epoch 2/2
811/811 ━━━━━━━━━━━━━━━━━━━━ 143s 172ms/step - accuracy: 0.1135 - loss: 5.7786 - val_accuracy: 0.0904 - val_loss: 6.8515


In [26]:
model.save('keras_next_word_model.h5')
pickle.dump(history, open("history.p", "wb"))
model =load_model('keras_next_word_model.h5')
history = pickle.load(open("history.p", "rb"))

In [31]:
def prepare_input(text):
    x = np.zeros((1, WORD_LENGHT, len(unique_words)))
    for t, word in enumerate(text.split()):
        print(word)
        x[0, t, unique_words_index[word]] = 1
    return x
prepare_input("How are you ".lower())

how
are
you


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

In [33]:
def sample(preds, top_n=3):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)

    return heapq.nlargest(top_n, range(len(preds)),preds.take)

In [35]:
def predict_completions(text, n=3):
    if text =="":
        return("0")
        x = prepare_input(text)
        preds = model.predict(x, verbose=0)[0]
        next_indices = sample(preds, n)
        return [unique_words[idx] for idx in next_indices]    

In [37]:
q = "Do your work by your own imstead of depending on someone"
print("correct sentence:",q)
seq = " ".join(tokenizer.tokenize(q.lower())[0:5])
print("sequence: ",seq)
print("next possible words: ",predict_completions(seq, 5))

correct sentence: Do your work by your own imstead of depending on someone
sequence:  do your work by your
next possible words:  None
